In [2]:
# IMPORTS
import cvxpy as cp
import numpy as np
import numpy.linalg
import numpy.matlib
from numpy.matlib import repmat
import csv   

from sklearn.preprocessing import PolynomialFeatures

from scipy.linalg import lstsq
from scipy.integrate import odeint

import pandas as pd

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

####################### FUNCTIONS #######################

# Simulate dynamical system

#Van der Pol oscillator
def vanderpol(x, t, gamma=1, kappa=1, epsilon=2):
    return [
            x[1],
            -kappa * x[0] - gamma * x[1] - epsilon * x[1] * x[0] ** 2
            ]

gamma=1
kappa=1
epsilon=2


####################### GENERATE DATA #######################

# Simulate dynamics

t0, tf = 0, 2.2  # start and end
dt = 0.01  # time step
n_times = int(np.floor((tf-t0)/dt)+ 1)  #Number of time instances

n_states = 2 #Number of state variables
t_span = np.linspace(t0, tf, n_times)
t = t_span

x0 = [1, 0]
X = odeint(vanderpol, x0, t_span)
x1 = X[:,0]
x2 = X[:,1]

#Compute true derivatives
dX = np.zeros((len(t_span),n_states))
dX[:,0] = x2
dX[:,1] =  -kappa * x1 - gamma * x2- epsilon * x2* x1** 2

#Corrupt states by adding noise --> Observation model y(t) = x(t) + e(t)
sigma = 0.0001
n_samples = 100

filepath = 'data_files/'
for state in range(n_states):
            
        E = sigma * np.random.randn(n_samples, n_times)#Additive zero-mean white noise (Assumed Gaussian)
        Y = repmat(X[:,state].T,n_samples,1) + E         

        filename = 'VanderPol_samples_noise_' + str(sigma) + '_Y' + str(state+1) + '.csv'
        # writing to csv file   
        with open(filepath+filename, 'w') as csvfile:   
            # creating a csv writer object   
            csvwriter = csv.writer(csvfile)   

            # writing the data rows   
            csvwriter.writerows(Y)

dynamics = np.concatenate(([t],X.T,dX.T), axis = 0)
filename = 'VanderPol_dynamics.csv'
# writing to csv file   
with open(filepath+filename, 'w') as csvfile:   
    # creating a csv writer object   
    csvwriter = csv.writer(csvfile)   

    # writing the data rows   
    csvwriter.writerows(dynamics)